# Streaming Analytics for Life Event Prediction 
This is an IBM Cloud Pak for Data Industry Accelerator project. 

This accelerator project guides you through an example which uses the IBM Streaming Analytics service in Cloud Pak for Data to analyze a stream of continuously arriving events and generates alerts as significant new predictions occur.  The project uses an event scoring model created in the Life Event Prediction Industry Accelerator available with CloudPak for Data version 2.1.0.1.

## Function overview
The Streaming Analytics for Life Event Prediction accelerator project includes and uses the scoring functions of a Life Event prediction (LFE) model. This LFE model takes as input a list (history) of one customer's interactions with a fictional financial services company. The LFE model generates predictions whether in the near future that customer will:
* Purchase a home
* Relocate to a new address

More information on how the LFE model was created and trained using machine learning can be found in the documentation for the Life Event Prediction Industry Accelerator available as an an add-on in Cloud Pak for Data versions 2.1.0.1 and later.

This Streaming Analytics for LFE accelerator shows how to: 
* Score a stream of arriving customer events **as they occur**
* Write the scores to a database enabling offline analysis
* Decide which new scores are **significant**
* Push the significant scores to a Kafka event stream enabling **automated proactive handling** of significant events
* Utilize an R Shiny dashboard to display recent significant scores   

The Streaming Analytics accelerator uses the IBM Streams add-on provided in Cloud Pak for Data. Each major component of this accelerator is provided in the form of a self documenting jupyter notebook. 

The Demonstration dashboard is an R Shiny application service deployed in Cloud Pak for Data. 



## Architectural Diagram

An overview of the Streaming analytics solution is provided in the diagram below. 
![fig Overview](../misc/images/strlfig1.png)


## Steps in working through the project

This README notebook will provide guidance on the following steps for working through the accelerator:

1. Address the prerequisites for running this project [prereqs_step](#step2):
     * You will need admin authority in your Cloud Pak for Data instance to configure connections to remote data resources
     * The Streams Cloud Pak for Data add-on must be installed and configured  
     * A relational database service must be provisioned for the LFE_SCORES table  
     * An IBM Event Streams (Kafka) service for the SIGNIFICANT_LFE_SCORES topic must be provisioned 
2.  Configure the connection to an LFE_SCORES relational table from your Cloud Pak for Data instance - [table setup step](#step3)
3.  Create and configure a SIGFNIFICANT_LFE_SCORES Kafka topic - [topic setup step](#step4)
4.  Configure and deploy services to subscribe to streaming analytics results and to support a display dashboard - [service deploy step](#step5)
     *  Create a tagged project release of the project code in the Cloud Pak for Data repository
     *  Deploy a python script based web service to significant events published by the streaming analytics to EventStreams
     *  Deploy an R Shiny dashboard application service for "business users" of the life event prediction streaming analytics 
5.  Start the streaming analytics simulation - [streaming analytics step](#step6)
     * Run the Streaming_analytics_lfe_pipeline notebook to start the streaming analytics simulation      
6.  Observe streaming analytics results in a business-user oriented dashboard - [use-dashboard-step](#step7)
     * Use the sample R Shiny Dashboard  
7.  Optional generalization - [optional-steps](#step8)

<a id='step2'></a>

## Step 1: Prerequisites  
Prerequisites to execute this Streaming Analytics for Life Event Prediction accelerator include: 
*   The owner or administrator of your Cloud Pak for Data instance must authorize your userid with admin privilege. 
    * If you do not have this, the options to see connections, organize data and administer deployments will not appear in the commands pull down in the main dashboard page.
*   Your CloudPak for Data instance must have a installed deployed “Streams” Add-On instance 
    * Documentation or this available at: [Deploying and downloading the IBM Streams add-on for Cloud Pak for Data](https://www.ibm.com/support/knowledgecenter/SSQNUZ_current/com.ibm.icpdata.doc/streams/download.html)
*   There must be a relational database service which is network reachable from your Cloud Pak for Data instance, and you must have userid and password with create table, drop table,  read and insert privilege  for a schema in this database
    *  Provisioning a Db2 Warehouse service on IBM Cloud meets this requirement. Setting up such a service is documented in [IndustryAccelerators-RemoteData](https://github.com/IBM-ICP4D/IndustryAccelerators-RemoteData)  
    *  The cell [Table setup step](#step3) in this notebook provides step by step instructions on configuring a connection from your Cloud Pak for Data instance to a remote relational table. This enables the Streaming Analytics simulation to write model scores into the LFE_SCORES table as they occur.  
*    There must be a Kafka Service which is network reachable from your Cloud Pak for Data instance with a topic SIGNIFICANT_LFE_SCORES. You must have credentials giving publish and subscribe authority to this topic.	      
     * The cell [topic setup step](#step4) in this notebook explains how to provision an IBM Event Streams Service in IBM Public Cloud, configure a topic meeting the above requirement, and obtain the credentials needed by the Streaming Analytics simulation
     

<a id='step3'></a>

## Step 2: Remote Relational Table  LFE_SCORES Setup 
Here we assume that a relational database service has been provisioned as described in Step 2. We will illustrate specific actions with diagrams from a Db2 Warehouse on IBM Cloud instance.

### 2.1  Create a connection from Cloud Pak for Data to the database service 
You will need to configure a connection from your Cloud Pak for Data instance to the relational database. 
*  From the main Cloud Pak for Data dashboard select _'connections"_ from the command pull down in the top left of the title bar 
    * The resulting page lists the currently defined connections
*  Select _'add connection'_. This will bring up the new connection page 
*  Enter a connection name and description.  Set the connection type to be Db2 or Db2 Warehouse on Cloud 
*  This will extend the form with additional lines:
   * Hostname - the TCP/IP address of the remote host
   * The port for the relational data service; in the case of a remote Db2 or Db2 Warehouse on Cloud the port is 50000
   * The Database name ; BLUDB in the case of remote Db2
   * Userid and password for accessing this remote relational data service.  The tutorial at [IndustryAccelerators-RemoteData](https://github.com/IBM-ICP4D/IndustryAccelerators-RemoteData) provides more information on how to obtain these value for the remote data service in Db2 WOC on IBM Cloud.  
*  When these fields are entered, select _'validate'_ to test the connection 
*  If this succeeds the connection information is okay and you will be able to save the newly defined connection from your Cloud Pak for Data instance to the remote relational database service. 

### 2.2   Identify this connection as a datasource to the Streaming Analytics for LFE project 
Next, the connection defined to your Cloud Pak for Data instance must be defined as a data source in the IndustryAccelerator-StreamingAnalytics project. From the main Cloud Pak for Data dashboard select _'projects'_ from the command pull-down 
*  From the list of projects, select this project 'IndustryAccelerator-StreamingAnalytics'
*  Select _'Data Sources'_ 
You must add the connection to a remote relational database service we defined above as a data source to it available for this project to use. 
*  Select _'Add data Source'_ - which will bring up a list of connections known to you Cloud Pak for Data instance. 
*  Select the connection you defined above by name - click _'next'_
   *  This will bring up a screen with some connection information. At the bottom of this form there are fields for user id and password. Make sure these are filled out. 
   *  The userid and password information needed in a specific project could be different than those used to set up the instance wide connection 
*  When all fields are entered  click 'Test Connection' if this succeeds, you will be able to select 'add connection' 
*  At this point the _'data sources'_ tab in the IndustryAccelerator-StreamingAnalytics project page will list the new data source 

### 2.3   Identify the LFE_SCORES table as a dataset within the datasource
Next we must identify a specific table LFE_SCORES in the remote data source as the table to be used to store scores from the streaming analytics.
*  In the IndustryAccelerator-StreamingAnalytics project screen, under the assets tab, select _'datasets'_ 
   * This will bring up a list of datasets defined to the project, initially all local datasets imported with the project files. 
*  Select _'Add Dataset'_ and then click on _'Remote Dataset'_
   *  The datasource pointing to the remote relational data connection you configured above will be available as a pulldown entry under _'select a data source'_
   *  Make LFE_SCORES the remote dataset name 
   *  A text description can be entered 
   *  Set the SQL object type to _'Table'_
   *  A schema name is required.  In the case of the remote service being Db2 WOC this will be a userid in the form _DASHxxxx_ 
   *  The Table name must be typed in at the bottom of the form 
*  At this point you will be able to add the new (Remote) data set to the project 

The relational table does not have to exist for this dataset definition to be made, since this is an output table. The table will be created by a cell in the lfe-Streaming_pipeline notebook. Any table left over from a previous execution of the streaming analytics simulation is dropped before this is done. 

<a id='step4'></a>

## Step 3  Setup SIGNIFICANT_LFE_SCORES  as an EventStreams (Kafka) topic
In this section we describe how to:
*  Provision a Kafka event service using Event Streams in IBM Cloud 
*  Create and configure a topic SIGNIFICANT_LFE_SCORES in this service where significant scores from the LFE model are published
*  Create a set of credentials to access this Event Streams topic
*  Import credentials to access that topic into the IndustryAccelerator-StreamingAnalytics project in Cloud Pak for Data

### 3.1 Provisioning  an Event Streams service  in the IBM Cloud
The Streaming Analytics for LFE project can publish significant scores to any available Kafka service. 
Here we describe how to provision and configure a Kafka service using Event Streams in the IBM Cloud. 
*  Login to an IBM Cloud Account [IBM Cloud Account](https://cloud.ibm.com/login) 
   * This will bring you to cloud _'Dashboard'_ page.  
*  On the 'Dashboard' page select _'Create resource'_.  This will bring you to the _'Catalog'_ page.
*  In the filter box at the top if the page type: _'Event Streams'_
*  Click on the _'Event Streams tile'_.  This will open the Event Streams service creation page.
*  The _'Service name:'_ box will be populated with a generated name for the _'Event Streams'_ service. In addition, the _'Lite'_ _'PLAN'_ will be selected. Press _'Create'_ to proceed with the creation of your _'Event Streams'_ service.  
*  Upon completion of the creation of your service, you'll be brought to your event stream management page.

The next step is to create the topic, 'SIGNIFICANT_LFE_SCORES', where significant live event scores and triggers for automated proactive actions are published.

### 3.2 Creating the SIGNIFICANT_LFE_SCORES topic
Event Streams communications channel identifiers are called _topics_.  To create a 'SIGNIFICANT_LFE_SCORES' topic on your newly created event streams service:
*  Click on the _'Create a Topic'_ tile from the management page of the event Streams service 
*  In the _'Topic name'_ cell enter *'SIGNIFICANT_LFE_SCORES'* as the name. Select _'Next'_ to proceed on to the _'Create topic Partitions'_ page.
*  No changes are needed on the partitions page (default 1 partition is sufficient for a non production service) ; select _'Next'_
*  No changes are needed on the message retention page (default 1 day retention will be sufficient for a steaming analytics demonstration); select _'Create Topic'_
*  When the topic is created you will be returned to the Event Streams management page. 
   *  A pop-up will appear stating that the topic creation has been requested
   *  If necessary refresh the page and check under the _'Topics'_ tab whether SIGNIFICANT_LFE_SCORES is listed

### 3.3  Creating  a credential for the SIGNIFICANT_LFE_SCORES topic 
The next step is to create a credential for this topic which will be used by the streaming_analytics_lfe_pipeline notebook when it publishes significant scores to the topic 
*  From the IBM Cloud Event Streams management page, in the left sidebar, select _'service credentials'_ to bring up the credentials page
*  If no credential is listed, click on _'New credential'_ to create one
   *  Existing credentials can also be viewed using the _'View credential'_ pulldown in the credential summary line 
*  Click on the _'Copy'_ icon in top right of the opened credential entry to copy JSON content to your copy/paste buffer.  

### 3.4  Saving the Topic credential to a file in Cloud Pak for Data 
The credential now saved in the copy/paste buffer of your workstation will now be saved as a file in your Streaming Analytics for LFE project in Cloud Pak for Data. 
This file will be in a known location where it will act as a remote data resource for notebooks in the project.
*  Return to the tab in your browser (connected to Cloud Pak for Data) with a Streaming Analytics for LFE project page.
*  In the upper right hand corner of the page, select the _'Launch Terminal with Python ..'_ icon (icon of a screen with a '>' symbol).
   * This will start a Jupyter terminal and bring up a command line window with a prompt 
*  Type in following command to create the Event Streams json file and wait for input:
```bash
cat > $DSX_PROJECT_DIR/datasources/credentials/eventstreams_$USER.json 
```
     
     
*  Paste in the your copy/paste buffer with your captured credential.
*  Type ^d (Ctrl_d) to write the contents and return to the terminal prompt. You can look at the contents of the newly created file with:
```bash
cat $DSX_PROJECT_DIR/datasources/credentials/eventstreams_$USER.json 
```

If you have already created the LFE_SCORE relational table remote dataset, you will be able to check that the datasources/credentials folder has separate json credential files to access the SIGNIFICANT_LFE_SCORES topic in Event Streams and the LFE_SCORES table in the remote relational database service.  

This completes the setup of the Event Streams Kafka service with a SIGNIFICANT_LFE_SCORES topic and configuration and import of credentials into the Streaming Analytics for LFE project in Cloud Pak for Data so that the the streaming_analytics_for_lfe pipeline notebook can publish scores to this topic.

<a id='step5'></a>

##  4. Deploying  Services  to retrieve results and display in Dashboard
Here we describe how to configure and deploy services in your Cloud Pak for Data instance which can be called to:
*   Fetch recent significant scores from the Streaming Analytics simulation 
*   Support display of recent scores and other information  about progress of the Streaming Analytics in an R Shiny Dashboard. 

### 4.1 Creating a tagged project snapshot for use in service deployments 
Deployments are created using a committed snapshot of analytics project code state in the Watson Studio master repository labelled this with a source code tag. 
To proceed with the service deployments below, you must first commit the IndustryAccelerator-StreamingAnalytics project into "Git" and tag the project image.  
*  Navigate to your 'streaming analytics for LFE' notebook by opening the navigation sidebar (using the _'lines'_ icon at left end of title bar), and selecting _'projects'_
    * Open the analytics project where you have imported the materials 
*  Using the _'git actions_' icon in the project title bar, click to pull down the list of available commands and select _'commit'_. 
    * The commit page identifies the repository, will show you which files are modified and require you to add a line of explanation 
    * If there are no changes to commit, add a new empty file to the project OR use commit history to add the tag we want to a previous commit of the project 
*  After successful commit use the _'git actions'_ list of commands to do a _'push'_
    * The push page includes a field where you can enter a tag identifying this project code state
*  Remember the source code tag for use in deployments below 
    * Or you can find it again later using the _'commit history'_ command in the _'git actions'_ list 
    
### 4.2  Create a project release  -  from a named project and source tag 
* From any Cloud Pak for Data page, open the navigation sidebar. Select 'Manage Deployments', to enter the 'Project releases' page.
    * The returned page will list any project releases already defined and visible to you in this Cloud Pak for Data instance. The list may be empty.
*  Use the "add project release" button in the header of the list to create a new project release with code you will deploy as services 
*  In the add project release page:
    * Select the "From IBM Cloud Pak for Data" tab  - because in step 4.1 above you committed to the Watson Studio Master Repository
    * Enter a Project Release name  - typically identifying project, date, or use i.e as test, production etc 
    * Enter a route - this string is embedded into urls for services deployed from the release
    * The source Project is the project you are working from and committed in step 4.1. Scroll through the pull down list of repository projects and pick yours
    * Tag is the identifying string you set in the push page in step 4.1
*  There are restrictions on the length of Project Release names and Route strings; project release names must be lower case only
*  When all fields on the page are format validated, the _'Create'_ button is activated and the new project release can be created  
*  Successful create will open a project release dashboard page
*  The > icon in the header bar of this page can be used to launch the project release

### 4.3  Configure and deploy a python service to retrieve data from the SIGNIFICANT_LFE_SCORES topic  
We need to deploy a service which will connect to Event Streams and subscribe to the SIGNIFICANT_LFE_SCORES topic so that it can make recent significant scores available to callers of the service. The logic for the service is implemented as a Python script eventstream.py visible as a script in the Streaming Analytics for LFE project. Python was motivated by the availability of python libraries for connecting and subscribing to a Kafka topic.

* Select your newly created project release's icon (3 perpendicular dots) to reveal the 'Launch' option, select this and the following confirmation 'Launch' window.
* A successful launch is indicated by a 'green' dot to the left of the project name, select the project name to enter the project release dashboard.
* Select the 'Assets' menu item on the dashboard.
* Select the 'eventstream.py' asset in the left column to enter the 'eventstream.py' deployment page.
* You are ready to start the 'web service', which requires two configuration variables EVENTSTREAMS_CREDENTIALS' and EVENTSTREAMS_TOPIC'. The EVENTSTREAMS_CREDENTIALS should be copied into your paste buffer with the following steps.
  * Open a new browser window
  * Login to your [IBM Cloud Account](https://cloud.ibm.com/login) to enter your 'Dashboard' page, select 'Services'
  * Select your provisioned 'Event Streams..' service to enter your 'Event Streams' management page. 
  * Select 'Service Credentials' and expand 'View credentials'. Select the copy icon, which will move the credentials into your paste buffer.
  * You may close this browser window and return to the 'eventstream.py' deployment page.
* Select 'web service' of the 'eventstream.py' deployment to enter the "Deploy eventstream.py as a web service" page.
* In the 'Name' field enter 'eventstream'
* In the 'Web service environment' pulldown, select 'Python 3.6 - Script as a Service'
* You will be adding two 'Environment variables', select '+' to add 2 input fields.
  * Enter 'EVENTSTREAMS_CREDENTIALS=' and paste your copy/paste buffer with the credentials immediately to the right of the equal sign. Do not include the single quotes. 
  * In the second field enter 'EVENTSTREAMS_TOPIC=SIGNIFICANT_LFE_SCORES', again do not include the single quotes. 
* Select the _'Create'_ button, you will be brought to the newly created 'eventstream' endpoint's screen.

### 4.4 Deploying the dashboard service 
Here we describe how to deploy the R SHINY dashboard for the project as a deployed service in your Cloud Pak for Data cluster. Users will be able to connect using a browser and see a dashboard application which models how the Life Event Prediction Streaming Analytics could be made available to business users. 

The python service for significant scores and he dashboard service could be deployed from different project releases or the same project release.

* From any Cloud Pak for Data page, open the navigation sidebar. Select _'Manage Deployments'_, to enter the _'Project releases'_ page.
* In the list of project releases, select the project release created in step 5.2 (or the project release with code for the R Shiny Dashboard if that is different)
* In the page for that project release select the _'datasources'_ tab
   * That will list datasources. There should be a line for the datasource you defined for to the remote relational data service you configured in step 3.2
* Select _'Edit Credentials'_ for this datasource using dots at end of the row to open available commands
* In the resulting credentials page, enter Userid and password as needed to access the remote relational data service, then click _'save'_
* In the project release page, select the _'assets'_ tab to pick the specific asset we will deploy as a service  
* 'ALL' may have a long list of assets. You can use up/down to filter and list R shiny apps only 
* Select the _'LifeEventPredictionDashboard'_ from the list
   * The right side of the page will list any existing deployments of that asset 
* To create a new deployment of the asset, select the _'+ app'_ button in the header of the list 
* In the new deployment page, enter the deployment name
   * The url for the new service is displayed and there are option to limit resources used by the deployment
   * Select _'create'_ to deploy the new service returning you to the deployment list for this asset 
   

### 4.5 Launching and Enabling  the deployed services 
For both the Python Event Service and the R Shiny Dashboard service, the following steps are needed to make the service visible to users.
The commands are available in the deployed services dashboard(s) for the project release(s) use - in the ... command pull down at right of line
*  The service must be enabled
   *  This can take a minute or so.  Page refreshes will display the service as 'enabled' when the operation is complete
*  Intended users of the service need to be authorized as 'members'
   *  The deployer is automatically a member; others need to be explicitly added using the _'members'_ tab in the deployment page  
*  The URL to use when requesting the service can be retrieved using the _'share endpoint info'_ command on the deployment line (... command pull down)
   * This also copies the url into your copy/paste buffer

<a id='step6'></a>

## 5. Start the Streaming Analytics Simulation 
At this point everything is installed, configured, and ready to observe the results of streaming analytics processing on a simulated stream of continuously arriving customer events. 
The streaming analytics simulation is started by opening and executing all cells in the lfe-streaming-analytics-pipeline notebook. 
The processing steps in this notebook are:
*  Configuration variables setup - you can change code here to adjust to your setup 
*  Python imports 
*  Streams instance connection 
*  Cancel your existing streams jobs 
   *  Before you start a new streaming Analytics LFE simulation, any jobs from a previous run associated with your Cloud Pak for Data Userid are cancelled
*  Database connection initialized and LFE SCORES table reset
   * To allow for repeated streaming analytics runs, the table with LFE Scores from a previous run is dropped and recreated 
* Streams operators are defined: 
   * To store in memory the 'up until now' history of each customers interactions are used by the LFE scoring functions
   * Logic to identify which new scores represent a change from previous scores 'with business significance'
* The flow(topology) for each Streams job is defined (in python) 
   * Record Score History
   * Publish Significant Events
   * Significant score event detection / generation
   * Score Events 
   * Simulated Real-time customer interactions
* Each of these jobs is submitted to the Streams instance 
   * The first four are submitted in parallel. The Simulated Real-time customer interactions job is started last after the others are in place 
   * Progress bar shows progress of each job submission. Success is indicated by a returned jobid for the job.
* The health of submitted and started jobs is displayed. 

More detailed documentation of each of these steps is provided as comments in the notebook in each cell.

If the final cell displays 5 streams jobs in healthy state, you can assume that the streaming analytics simulation has started successfully. 
Some intermediate cells generate warning relating to 'insecure connections'. This does not represent a problem. 

Once the streaming analytics job has started, you can:
*  Open the deployed business dashboard to see how the Life Event prediction information could be presented to a business user in a R SHINY application
*  Run the explore-retrieve-saved-scores notebook to view scores written to the remote LFE SCORES table 
*  Run the explore-eventstreams_reader notebook to view significant score changes published to the SIGNIFICANT_LFE_SCORES topic 
*  Run the explore-streams_view notebook for views of specific data streams in the streaming analytics 

The Streaming Analytics for Life Event Prediction simulation runs with currently provided source data for several hours. It  reads a file of data on customer events which occurred over a period of years and feed these into the simulation at 2 events / sec to demonstrate how streaming analytics on continuously arriving data feeds is organized.

<a id='step7'></a>

## 6.  Observe streaming analytics results in a business-user dashboard
Results of the streaming analytics simulation can be observed in a deployed R SHINY application dashboard. The sample dashboard organizes and presents the results as would be expected by a business user. 
The dashboard service was deployed in step 5.4 before the real time simulation was started. 

The URL for the dashboard service is available from the project release -> deployments list using the _'share endopoint'_ command. Persons other than the original deployer will need to be added as members of the deployment to be authorized for this dashboard application. That can be done using the members tab on the deployment listing.

Pointing a browser to this url will bring up the dashboard.

The dashboard has tabs:
* Top clients
* Streaming significant events
* Real-time client activity 

The **real time client activity** tab picks up data from the LFE SCORES table where scores are written. If multiple remote datasets were defined in the Streaming Analytics project the user is able to select which one to use. You should select the dataset where your streaming analytics simulation is currently writing its Life Event Prediction scores - LFE_SCORES by default. This is the table of _all_ scores from the streaming analytics simulation so far. Credentials to read the table were stored in the deployed dashboard service. 

Clicking on the 'load events' tile displays the data currently in this table.

The **Streaming significant events** tab has an authenticate api step. 
Enter:
*  The hostname of the Cloud Pak for Data Login node
*  A userid authorized for that service 
*  Password for that userid 

This will bring up a list of event services. Select the deployed service which has access to the EventStreams topic where the current Streaming analytics 
simulation is publishing significant scores. The name will depend on how you deployed this service in step 5.3.

Clicking on the 'Load Events' tile  will display recent significant scores. A score is considered significant is it is the first score for this customer 
which has moved from being above a high likelihood (50%) to below a low probability (40%) or vice versa. The significant events list shows which transition
has been detected. 

Significant event would be used for automated customer outreach proactive process. As soon as a significant customer change has been detected, a text
message or email responding to the newly detected customer state can be sent.  

The **top clients** shows a list of selected client with profile and news information. Clicking on the 'load data' tile will show interactions
and scores up to the current time in the simulation and a plot of how these scores may have been significant. The two horizontal red lines in the plot
represent the high probability threshold(50%) and the low probability threshold(40%) respectively.

In this time plot for a customer, we show original time for the customer interaction events which are days and months old. The Streaming analytics simulation
runs with these events much faster, handling the overall stream for all customers at 2 events / second. This gives a demonstration of streaming 
analytics where change can be observed at a reasonable pace.


<a id='step8'></a>

## 7. Optional generalizations and extension
The Streaming Analytics for Life Event Prediction project can be generalized or extended in several ways. 
 * Change the logic/rules for which events are considered significant 
 * Connect to other event sources , and data stores
 * Modify the notebooks to perform streaming analytics with other Artificial intelligence / machine learning scoring models  
 
More detail on the benefits and issues in performing these generalization, and references for the project overall are provided in the developer-notes.ipynb notebook in the project 

**This project contains Sample Materials, provided under license.  
Licensed Materials - Property of IBM.  
© Copyright IBM Corp. 2019. All Rights Reserved.  
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.**
